In [1]:
import requests
import re 
import time
import json
from pymongo import MongoClient
import json
import random

client = MongoClient('localhost', 27017)

# let's get the list of DBs already defined
print(client.list_database_names())

# access token can be used till february 2024
access_token = "EAAKb83xTK40BO7gBDVcd3GoFsLMwJFEv1MrkbdbqpTKOVeA2a0GuZCnusq2SuGeD4IG2GjbeEV5OkV4dzHEN53KTRoDgnsKlsPLAIkuo2Yz7zdtEYRl3IvM4mHct3e6syQu0MjKt4wq3FViWZBkms1jGhxbFmOe8ZALYRQNfOBTOs4wH9I5v4kC"
instagram_account_id = 17841402187545471


graph_url = 'https://graph.facebook.com/v18.0/'
def get_hashtag_id(hashtag = '',instagram_account_id = '',access_token = ''):
    url = graph_url + 'ig_hashtag_search'
    param = dict()
    param['user_id'] = instagram_account_id
    param['q'] = hashtag
    param['access_token'] = access_token
    response = requests.get(url,param)
    response = response.json()
    
    hashtag_id = response['data'][0]['id']
    return hashtag_id

# retrieve name and id for a hasthag
def get_data_of_hashtag(hashtag_id = '',access_token=''):
    url = graph_url + hashtag_id
    param =dict()
    param['access_token'] = access_token
    param['fields'] = 'id,name'
    response = requests.get(url,param)
    response = response.json()
    return response

# recent media for a hashtag
def get_recent_media(hashtag_id='',access_token = '',instagram_account_id = ''):
    url = graph_url + hashtag_id + '/recent_media'
    param = dict()
    param['access_token'] = access_token
    param['user_id'] = instagram_account_id
    param['fields'] = 'caption,like_count,permalink,timestamp,media_type,comments_count,media_url'
    response = requests.get(url, param,timeout=90)
    response = response.json()
    print(response)
    return response

def get_top_media(hashtag_id='',access_token='',instagram_account_id=''):
    url = graph_url + hashtag_id + '/top_media'
    param = dict()
    param['access_token'] = access_token
    param['user_id'] = instagram_account_id
    param['fields'] = 'caption,like_count,permalink'
    response = requests.get(url, param)
    response = response.json()
    print(response)
    return response

def get_all_posts_since_24hours(hashtag_names, access_token, instagram_account_id):
    posts = []
    hashtag_ids = []
    wait_for_decreaseing_limits()
    for name in hashtag_names:
        hashtag_id = get_hashtag_id(name,instagram_account_id,access_token)
        hashtag_ids.append(hashtag_id)
    
    random.shuffle(hashtag_ids)
    
    for hashtag_id in hashtag_ids:
        # print to verify the id
        wait_for_decreaseing_limits()
        print("data for hashtag")
        print(get_data_of_hashtag(hashtag_id,access_token))
        posts_for_hashtag = get_all_posts_since_24hours_for_given_hashtag_id(hashtag_id,access_token, instagram_account_id)
        client['bda']['posts'].insert_many(posts_for_hashtag)
        posts = posts + posts_for_hashtag
    return posts


#def plot_most_common_hashtags_in_data()
def wait_for_decreaseing_limits():
    try:
        param = dict()
        param['access_token'] = access_token
        response = requests.get(graph_url+"me/",param,timeout=300)
        # Here we look at three metircs total time, total cpu time and call count that give us percentages  for their limit usages.
        # the code get's delayed one hour, till they reset
        usage = json.loads(response.headers['x-app-usage'])
        print(usage)
        if usage["total_cputime"] >= 90 or usage["total_time"] >= 90 or usage["call_count"] >= 90:
            print("sleeping")
            time.sleep(600)
    except requests.exceptions.Timeout:
        print("Timeout")

def get_all_posts_since_24hours_for_given_hashtag_id(hashtag_id,access_token,instagram_account_id):
    data = []
    count = 0
    wait_for_decreaseing_limits()
    recent = get_recent_media(hashtag_id, access_token,instagram_account_id)
    data = [] + recent["data"]
    while True:
        if not ( 'paging' in recent and 'next' in recent['paging']):
            break
        count = count + 1
        try:
            response = requests.get(recent['paging']['next'] ,timeout=300)
            if (count%50) == 0:
                wait_for_decreaseing_limits()
            
            recent = response.json()
            # we saw that sometimes there is at the last pages not data returned, but urls for new pages
            if'data'not in recent:
                break
            print(recent['data'][0]['timestamp'])
            data = data + recent['data']
        except requests.exceptions.Timeout:
            print("Timeout")
            print(recent)
            break
        except: #code to run if error occurs
            print("caught_it")
            print(recent)
    return data
    
    


['admin', 'bda', 'config', 'local']


In [39]:
wait_for_decreaseing_limits()


{'call_count': 22, 'total_cputime': 0, 'total_time': 9}


In [5]:
from pymongo import MongoClient
import json

client = MongoClient('localhost', 27017)

# let's get the list of DBs already defined
print(client.list_database_names())

['admin', 'bda', 'config', 'local']


In [3]:
# let's create a new DB - bda
database = client['bda']

# let's create a collection
posts = database.posts

In [2]:

while True:
    posts = get_all_posts_since_24hours(['war'],access_token,instagram_account_id)
    print(f"posts succesfull getted {len(posts)}")
    print(f"posts succesfull getted {client['bda']['posts'].count_documents({})}")


{'call_count': 19, 'total_cputime': 0, 'total_time': 43}
{'call_count': 20, 'total_cputime': 0, 'total_time': 43}
data for hashtag
{'id': '17843857138016461', 'name': 'war'}
{'call_count': 20, 'total_cputime': 0, 'total_time': 43}
{'data': [{'caption': 'ഇസ്രായേലിനെ ന്യായീകരിച്ച് വിദ്വേഷ പരാമർശങ്ങളുമായി ജോൺസൻ ആൻഡ് ജോൺസൻ മുൻ മേധാവി\n\nഇസ്രായേൽ നടത്തുന്ന ഫലസ്തീൻ കൂട്ടക്കൊലയെ ബൈബിൾ വചനം ഉദ്ധരിച്ചാണ് ജോൺസൻ ആൻഡ് ജോൺസൻ മുൻ വൈസ് പ്രസിഡന്റ് സാം മൽഡൊണാഡോ ന്യായീകരിച്ചത്\n.\n.\n.\n.\n.\n.\n.\n.\n#johsonandjohnson #sammaldonado #gazzawar #war', 'like_count': 4, 'permalink': 'https://www.instagram.com/reel/C1UUz4QqlA3/', 'timestamp': '2023-12-26T13:16:35+0000', 'media_type': 'VIDEO', 'comments_count': 0, 'media_url': 'https://scontent-mad2-1.cdninstagram.com/o1/v/t16/f1/m82/F447B0F08B5189C694F11B73837B0B86_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLmNsaXBzLnVua25vd24tQzMuNzIwLmRhc2hfYmFzZWxpbmVfMV92MSJ9&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=102&vs=1034397174484046_1277

I searched after the popularst hastags for the palätina isreal war. So I have searched 5 different hastags for the palästina side and 5 for isreal. For the palästina side I fount the hashtags #FreePalestine, #palistina, #savepalestine, #alquds (the holy city), gazaunderattack, #palestinewillbefree, #palastine. For the isreal side I have : #IsraelUnderAttack, #FreeIsrael, #israel, #proisrael (30000), #standwithisreal, #BringThemHomeNow. I will put all posts in one collection into a mongodb database.Also we take the hashtag #war to see a more broader overview of what is discussed under this hashtag.

In [3]:
# get data for hashtags
hashtags_for_palästina = ['freepalestine', 'palestine', 'savepalestine','palestinewillbefree','gazaunderattack','gaza']
hashtags_for_isreal = ['IsraelUnderAttack', 'prayforisrael', 'israel', 'proisrael', 'standwithisrael', 'BringThemHomeNow']
hashtags_for_general_war = ['war']

In [4]:
while True:
    posts = get_all_posts_since_24hours(hashtags_for_palästina+hashtags_for_isreal+hashtags_for_general_war,access_token, instagram_account_id)
    print(f"posts succesfull getted {len(posts)}")
    print(f"posts succesfull getted {client['bda']['posts'].count_documents({})}")
    #sleep 3 hours till next execution
    time.sleep(10800)


{'call_count': 20, 'total_cputime': 0, 'total_time': 41}
{'call_count': 25, 'total_cputime': 0, 'total_time': 43}
data for hashtag
{'id': '17841596584074200', 'name': 'israel'}
{'call_count': 27, 'total_cputime': 0, 'total_time': 43}
{'data': [{'caption': 'Für Amine war der Gaza-Streifen immer weit weg - sie wuchs als Tochter eines Palästinensers in Deutschland auf. Doch seit dem Terroranschlag der radikal-islamischen Hamas auf Israel wird sie immer wieder nach ihrer Meinung zum Nahostkonflikt gefragt und soll Stellung beziehen, obwohl sie nicht mehr Informationen habe als die anderen, sagt sie. Und dennoch: Aufgrund ihres palästinensischen Hintergrundes empfindet sie es als ihre Pflicht, zu vermitteln. Amine möchte Lehrerin werden und Kinder im Kampf gegen Antisemitismus aufklären. \n \nIn den Medien und der öffentlichen Diskussion vermisst sie oft die palästinensische Seite. "Das ist ungerecht und vermittelt Menschen mit palästinensischen Wurzeln das Gefühl, ausgegrenzt und benachtei

In [14]:
client['bda']['posts'].count_documents({})

65923

In [12]:
client['bda']['posts'].insert_many(posts)

In [13]:
client['bda']['posts'].

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bda'), 'posts')

In [12]:
client['bda']['posts'].count_documents({})

71012